## Consolidation Master File

##### Generate master file file from the list of files in the given directory

In [2]:
# Importing Libraries
import pandas as pd
import xlwings as xw
import os
import time

In [3]:
def lastRow(idx, workbook, col=1):
    """ Find the last row in the worksheet that contains data.

    idx: Specifies the worksheet to select. Starts counting from zero.

    workbook: Specifies the workbook

    col: The column in which to look for the last cell containing data.
    """

    ws = workbook.sheets[idx]

    lwr_r_cell = ws.cells.last_cell      # lower right cell
    lwr_row = lwr_r_cell.row             # row of the lower right cell
    lwr_cell = ws.range((lwr_row, col))  # change to your specified column

    if lwr_cell.value is None:
        lwr_cell = lwr_cell.end('up')    # go up untill you hit a non-empty cell

    return lwr_cell.row

In [ ]:
sheet1 = pd.DataFrame()
sheet2 = pd.DataFrame()
sheet3 = pd.DataFrame()
sheet4 = pd.DataFrame()
sheet5 = pd.DataFrame()
list_Sheets=['Update Allocation','Client Queries','Priority Updates','New Projects | News Review','Primary Support | OtherActivity']
fPath = input('Enter Folder Path')

for file in os.listdir(fPath):
    
    # Opening excel file
    wb = xw.Book(os.path.join(fPath,file))
    app = xw.apps.active
    
    eID = int(wb.sheets('Update Allocation').range('A1').value)
    eName = wb.sheets('Update Allocation').range('A2').value
   
    for sheet in list_Sheets:
        sht = wb.sheets(sheet)
        try:
            wb.sheets(sheet).api.ShowAllData()
        except:
            pass

        if sheet == list_Sheets[0]:
            
            sht1 = sht.range('A19:AE'+str(lastRow(sht,wb,1))).options(pd.DataFrame, index=False).value
            sht1.columns = ['Project ID','Parent Project ID ', 'Project Name', 'Profile Type', 'Project Value', 'Project Status',
                            'End Quarter', 'End Year', 'Region', 'Country', 'Published Date', 'Const. Start Date', 'Const. Quarter', 
                            'Ultimate Parent ID', 'Contact Opp.', 'NFU', 'Value Range', 'Age in days', 'Age in Months', 
                            'No. of Sub Projects', 'Stage', 'Priority', 'Allocated To ID', 'Allocated To Name', 'Status', 'Updated Date', 
                            'Update Type', 'Contacts Tagged', 'Contacts Created', 'Contacts Verified', 'Comments']
            
        elif sheet == list_Sheets[1]:
            sht2 = sht.range('A19:L'+str(lastRow(sht,wb,1))).options(pd.DataFrame, index=False).value
            sht2.columns = ['Date', 'Query ID', 'Project ID', 'Project Name', 'Activity Type', 'Contacts Tagged', 'Contacts Created', 
                            'Contacts Verified', 'Minutes', 'Comment', 'Emp ID', 'Emp Name']
            sht2['Emp ID'] = eID 
            sht2['Emp Name'] =eName
            
        elif sheet == list_Sheets[2]:
            sht3 = sht.range('A19:J'+str(lastRow(sht,wb,1))).options(pd.DataFrame, index=False).value
            sht3.columns = ['Date', 'Project ID', 'Project Name', 'Activity Type', 'Contacts Tagged', 'Contacts Created', 
                            'Contacts Verified', 'Comment', 'Emp ID', 'Emp Name']
            sht3['Emp ID'] = eID
            sht3['Emp Name'] =eName
            
        elif sheet == list_Sheets[3]:
            sht4 = sht.range('A19:I'+str(lastRow(sht,wb,1))).options(pd.DataFrame, index=False).value
            sht4.columns = ['Date', 'Emp ID', 'Emp Name', 'Project ID', 'Project Name / News Headline', 'Profile Type', 'Activity', 
                            'Contacts Tagged', 'Contacts Created']
            sht4['Emp ID'] = eID
            sht4['Emp Name'] =eName
            
        elif sheet == list_Sheets[4]:
            sht5 = sht.range('A19:L'+str(lastRow(sht,wb,1))).options(pd.DataFrame, index=False).value
            sht5.columns = ['Date', 'Emp ID', 'Emp Name', 'Other Activity', 'Assigned by', 'Time Spent in Min', 'Project ID', 
                            'Project Name / Activity Detail', 'Contacts Tagged', 'Contacts Created', 'Contacts Verified', 'Comment']
            sht5['Emp ID'] = eID
            sht5['Emp Name'] =eName
            

    # Closing excel file and application
    app.quit()
    time.sleep(.5)
    print(file)

    sheet1 = sheet1.append(sht1)
    sheet2 = sheet2.append(sht2)
    sheet3 = sheet3.append(sht3)
    sheet4 = sheet4.append(sht4)
    sheet5 = sheet5.append(sht5)

In [6]:
wb2 = xw.Book()
app = xw.apps.active

# Create new sheets
for sheet in list_Sheets:
    wb2.sheets.add(name=sheet)

# Delete default sheets
for sheets in ['Sheet1','Sheet2','Sheet3']:
    try:
        wb2.sheets(sheets).api.Delete()
    except:
        pass

# Copying DataFrame to sheets
for sheet in list_Sheets:
    sht = wb2.sheets(sheet)
    
    if sheet == list_Sheets[0]:
        sht.range('A1').value = sheet1
    elif sheet == list_Sheets[1]:
        sht.range('A1').value = sheet2
    elif sheet == list_Sheets[2]:
        sht.range('A1').value = sheet3
    elif sheet == list_Sheets[3]:
        sht.range('A1').value = sheet4
    elif sheet == list_Sheets[4]:
        sht.range('A1').value = sheet5

# Saving & closing excel file and application
wb2.save(os.path.join(fPath,'Master_Consolidation.xlsx'))
# wb2.close()
app.quit()